In [ ]:
import config
import code_improvement.code_execution as execution

from utils import prompt, tools
from langchain.prompts import ChatPromptTemplate

In [2]:
def determine_term_list():
    term_prompt = ChatPromptTemplate.from_messages(
        [("system", "You are a very helpful assistant."), ("user", "{input}")]
    )
    term_chain = term_prompt | config.llm

    response = term_chain.invoke(
        {
            "input": prompt.concept_determination_prompt.format(
                config.user_input_multimodal
            )
        }
    )
    term_list = response.content.split(", ")

    return term_list

In [3]:
def determine_URL_list(term_list):
    URL_list = []
    agent_executor = tools.create_agent(
        have_message_history=False, tool_list=[config.search], message_history=None
    )

    for term in term_list:
        print("Working on term: {} . . .".format(term))

        response = agent_executor.invoke(
            {
                "input": prompt.concept_searching_prompt.format(
                    config.user_input_multimodal, term
                )
            }
        )

        URL_list.extend(response["output"].split(", "))

    return URL_list

In [4]:
def generate_high_level_design(agent_executor):
    response = agent_executor.invoke(
        {"input": prompt.high_level_design_prompt.format(config.user_input_multimodal)},
        config=config.session_configuration,
    )

    return response["output"]

In [5]:
def generate_detailed_design(agent_executor):
    response = agent_executor.invoke(
        {"input": prompt.detailed_design_prompt.format(config.user_input_multimodal)},
        config=config.session_configuration,
    )

    step_list = response["output"].split("\n\n")

    return step_list

In [6]:
def generate_code_segment(agent_executor, step_list):
    code_list = []

    for step_description in step_list:
        print(
            "Now, we are generating code segment for step {}".format(
                step_description.split("\n")[0]
            )
        )

        try:
            response = agent_executor.invoke(
                {
                    "input": prompt.code_segment_prompt.format(
                        config.user_input_multimodal,
                        config.programming_language,
                        step_description.split("\n")[0],
                        step_description[step_description.index("\n") + 1 :],
                        config.programming_language,
                        step_description.split("\n")[0],
                    )
                },
                config=config.session_configuration,
            )
        except:
            continue

        code_segment = tools.sanitize_output(response["output"])
        code_list.append(code_segment)

    return code_list

In [7]:
def generate_final_code(agent_executor):
    response = agent_executor.invoke(
        {"input": prompt.code_combination_prompt.format(config.user_input_multimodal)},
        config=config.session_configuration,
    )

    return tools.sanitize_output(response["output"])

In [8]:
def execute_code(code, data_path):
    with open("generated_code.py", "w") as file:
        file.write(code)

    command = "python3 generated_code.py -i {}".format(data_path)
    status, program_output = execution.run(command, True)

    return status, program_output

In [9]:
# first determine a list of terms to be serached
term_list = determine_term_list()
print(
    "According to your requirements, we decide to search for the definition of the following terms:",
    term_list,
)

According to your requirements, we decide to search for the definition of the following terms: ['audio', 'depth camera', 'radar data', 'multimodal dataset', 'human activity recognition', 'PyTorch', 'numpy', 'training data', 'testing data', 'machine learning model', 'recognition accuracy']


In [10]:
URL_list = determine_URL_list(term_list)
print("Here is the URL list we crawled:")
for URL in URL_list:
    print(URL)

c:\Users\zjusl\AppData\Local\Programs\Python\Python310\lib\site-packages\langsmith\client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Working on term: audio . . .


> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'audio definition site:wikipedia.org'}`


We have searched the following information:
url: https://en.wikipedia.org/wiki/Audio
content: Audio most commonly refers to sound, as it is transmitted in signal form. It may also refer to:
url: https://en.wikipedia.org/wiki/Sound
content: Sound is a vibration that propagates as an acoustic wave through a transmission medium such as a gas, liquid or solid.
url: https://en.wikipedia.org/wiki/Audio_signal
content: An audio signal is a representation of sound, typically using either a changing level of electrical voltage for analog signals or a series of binary numbers
url: https://en.wikipedia.org/wiki/Audio_equipment
content: Audio equipment refers to devices that reproduce, record, or process sound. This includes microphones, radio receivers, AV receivers, CD players, tape recorders
url: https://en.wikipedia.org/wiki/Digi

In [11]:
document_list = tools.load_new_webpage(URL_list)
agent_with_chat_history = tools.create_agent(
    have_message_history=True,
    tool_list=[config.search, tools.create_context_retrieval_tool(document_list)],
    message_history=config.message_history,
)

We will further search information from these websites:
https://en.wikipedia.org/wiki/Audio
https://www.e-consystems.com/blog/camera/technology/what-are-depth-sensing-cameras-how-do-they-work/
https://www.ncei.noaa.gov/products/radar
https://www.weather.gov/phi/RadarSatellite
https://www.uniphore.com/glossary/multimodal-data/
https://www.owkin.com/a-z-of-ai-for-healthcare/multimodal-data
https://encord.com/blog/top-10-multimodal-datasets/
https://www.v7labs.com/blog/multimodal-deep-learning-guide
https://en.wikipedia.org/wiki/Multimodal_learning
https://en.wikipedia.org/wiki/Activity_recognition
https://github.com/pytorch/pytorch
https://pytorch.org/
https://en.wikipedia.org/wiki/NumPy
https://numpy.org/doc/2.2/user/whatisnumpy.html
https://en.wikipedia.org/wiki/Training,_validation,_and_test_data_sets
https://en.wikipedia.org/wiki/Test_data
https://www.lambdatest.com/learning-hub/test-data
https://www.geeksforgeeks.org/what-is-test-data-in-software-testing/
https://www.datprof.com/sol

c:\Users\zjusl\AppData\Local\Programs\Python\Python310\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')
c:\Users\zjusl\AppData\Local\Programs\Python\Python310\lib\site-packages\langsmith\client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [12]:
high_level_design = generate_high_level_design(agent_with_chat_history)
print("We first provide the algorithm design outline:\n{}".format(high_level_design))



> Entering new AgentExecutor chain...

Invoking: `context_document_search` with `{'query': 'multimodal human activity recognition using PyTorch'}`






Activity recognition - Wikipedia



























Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload fileSpecial pages



















Search











Search






















Appearance
















Donate

Create account

Log in








Personal tools





Donate Create account Log in





		Pages for logged out editors learn more



ContributionsTalk




























Contents
move to sidebar
hide




(Top)





1
Types




Toggle Types subsection





1.1
Sensor-based, single-user activity recognition






1.1.1
Levels of sensor-based activity recognition










1.2
Sensor-based, multi-user activity recognition



In [13]:
detailed_design_step_list = generate_detailed_design(agent_with_chat_history)

print("Based on the high level design, we provide the detailed design as:")
for element in detailed_design_step_list:
    print(element, "\n")



> Entering new AgentExecutor chain...

Invoking: `context_document_search` with `{'query': 'multimodal human activity recognition PyTorch'}`






Activity recognition - Wikipedia



























Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload fileSpecial pages



















Search











Search






















Appearance
















Donate

Create account

Log in








Personal tools





Donate Create account Log in





		Pages for logged out editors learn more



ContributionsTalk




























Contents
move to sidebar
hide




(Top)





1
Types




Toggle Types subsection





1.1
Sensor-based, single-user activity recognition






1.1.1
Levels of sensor-based activity recognition










1.2
Sensor-based, multi-user activity recognition









In [14]:
code_segment_list = generate_code_segment(
    agent_with_chat_history, detailed_design_step_list
)

Now, we are generating code segment for step Step 1: Loading Data


> Entering new AgentExecutor chain...

Invoking: `context_document_search` with `{'query': 'numpy load npy files'}`






NumPy - Wikipedia




































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload fileSpecial pages



















Search











Search






















Appearance
















Donate

Create account

Log in








Personal tools





Donate Create account Log in





		Pages for logged out editors learn more



ContributionsTalk




























Contents
move to sidebar
hide




(Top)





1
History




Toggle History subsection





1.1
matrix-sig








1.2
Numeric








1.3
Numarray








1.4
NumPy










2
Features




Toggle Features subsection





2.1


In [15]:
final_code = generate_final_code(agent_with_chat_history)



> Entering new AgentExecutor chain...
```python
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import argparse
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau


# Load multimodal data from npy files and verify their integrity.
def load_multimodal_data(dataset_path):
    train_audio_data = np.load(f"{dataset_path}/train_audio.npy")
    train_depth_data = np.load(f"{dataset_path}/train_depth.npy")
    train_radar_data = np.load(f"{dataset_path}/train_radar.npy")
    train_label = np.load(f"{dataset_path}/train_label.npy")
    test_audio_data = np.load(f"{dataset_path}/test_audio.npy")
    test_depth_data = np.load(f"{dataset_path}/test_depth.npy")
    test_radar_data = np.load(f"{dataset_path}

In [ ]:
dataset_path = input("Please input the path to your dataset: ")

In [ ]:
print("Next, we will proceed to the supervised code generation and modification stage.")

for epoch in range(config.epoch_number):
    status, program_output = execute_code(final_code, dataset_path)
    while status != 0:
        response = agent_with_chat_history.invoke(
            {
                "input": prompt.correct_grammar_prompt.format(
                    config.user_input_multimodal, program_output
                )
            },
            config=config.session_configuration,
        )
        final_code = tools.sanitize_output(response["output"])

        status, program_output = execute_code(final_code, dataset_path)

    response = agent_with_chat_history.invoke(
        {
            "input": prompt.improve_code_prompt.format(
                config.user_input_multimodal, program_output
            )
        },
        config=config.session_configuration,
    )
    high_level_design = response["output"]
    detailed_design_step_list = generate_detailed_design(agent_with_chat_history)
    code_segment_list = generate_code_segment(
        agent_with_chat_history, detailed_design_step_list
    )
    final_code = generate_final_code(agent_with_chat_history)

Next, we will proceed to the supervised code generation and modification stage.
************** START **************
Traceback (most recent call last):
File "/home/iot/Documents/AutoNLP/generated_code.py", line 226, in <module>
main(args.input)
File "/home/iot/Documents/AutoNLP/generated_code.py", line 190, in main
features = extract_features(segments)
File "/home/iot/Documents/AutoNLP/generated_code.py", line 101, in extract_features
return np.array(features)
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (8578, 15) + inhomogeneous part.
************** FAILED **************


> Entering new AgentExecutor chain...
```python
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import argparse
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
fr

In [73]:
# search for all the generated comprehensive code
combined_code_list = []

for index in range(0, len(config.message_history.messages), 2):
    question = config.message_history.messages[index].content

    if (
        "Based on the user's problem and all the generated code segments for each module, please constructively integrate them to form a comprehensive and cohesive piece of code. Please also embed detailed comments."
        in question
    ):
        combined_code = tools.sanitize_output(
            config.message_history.messages[index + 1].content
        )
        combined_code_list.append(combined_code)

In [74]:
program_output_list = []

for index in range(len(combined_code_list)):
    status, program_output = execute_code(combined_code_list[index], dataset_path)
    program_output_list.append(program_output)

************** START **************
Traceback (most recent call last):
File "/home/iot/Documents/AutoNLP/generated_code.py", line 227, in <module>
main(args.input)
File "/home/iot/Documents/AutoNLP/generated_code.py", line 191, in main
features = extract_features(segments)
File "/home/iot/Documents/AutoNLP/generated_code.py", line 102, in extract_features
return np.array(features)
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (8578, 15) + inhomogeneous part.
************** FAILED **************
************** START **************
/home/iot/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunct

In [75]:
running_result = ""

for index in range(len(program_output_list)):
    running_result = "{}\n\n<version_{}>\n{}\n</version_{}>\n\n".format(
        running_result, index + 1, program_output_list[index], index + 1
    )


response = agent_with_chat_history.invoke(
    {"input": prompt.final_decision_prompt.format(running_result)},
    config=config.session_configuration,
)



> Entering new AgentExecutor chain...
2

> Finished chain.


In [76]:
version_index = int(response["output"])
final_code = combined_code_list[version_index - 1]

print("Here is the final version of code to solve your problem.")
print("============================================")
print(final_code)
print("============================================")

Here is the final version of code to solve your problem.

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import argparse
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau


# Load multimodal data from npy files and verify their integrity.
def load_multimodal_data(dataset_path):
    train_audio_data = np.load(f"{dataset_path}/train_audio.npy")
    train_depth_data = np.load(f"{dataset_path}/train_depth.npy")
    train_radar_data = np.load(f"{dataset_path}/train_radar.npy")
    train_label = np.load(f"{dataset_path}/train_label.npy")
    test_audio_data = np.load(f"{dataset_path}/test_audio.npy")
    test_depth_data = np.load(f"{dataset_path}/test_depth.npy")
    test_radar_data = np.load(f"{datas

In [ ]:
response = agent_with_chat_history.invoke(
    {
        "input": prompt.user_documentation_prompt.format(
            config.user_input_multimodal, final_code
        )
    },
    config=config.session_configuration,
)



> Entering new AgentExecutor chain...
# User Documentation for the Human Activity Recognition System

## Introduction

### User's Problem
The user aims to develop a human activity recognition (HAR) system using the Wireless Sensor Data Mining (WISDM) dataset. This dataset includes accelerometer data collected from smartphones, which can be used to identify human activities. The task involves preprocessing the WISDM data, dividing it into training and testing parts, constructing a machine learning model for activity recognition, and finally evaluating the model's average recognition accuracy on the test dataset.

### The Code
The provided code is a Python script that implements a HAR system using the PyTorch framework. The script is designed to run on systems with a powerful GPU and uses a combination of convolutional neural networks (CNNs), long short-term memory (LSTM) networks, and attention mechanisms to classify accelerometer data into activity categories.

## Getting Started

##